# Part 5.1: Post-processing
## Extract and convert geocoordinates from predicted masks

Since the model delivered very good results during the test phase, the predicted masks can be used for further analysis. In this first part of the post-processing phase, the pixel coordinates are extracted with the help of the rasterio library. 

Contrary to the previous data set of pedestrian crossings, the TIF files in this data set do not use the LV95 coordinate system! The reference system for the coordinates of the refuge island is EPSG:3857 (WGS 84 coordinates in Pseudo-Mercator -- Spherical Mercator form)
> This projected coordinate system is used for rendering maps in Google Maps, OpenStreetMap, etc.

The extracted coordinates are stored in a geojson file for later comparison.

In [2]:
import rasterio as rio
import tifffile
import cv2
from matplotlib import pyplot
from fastai.vision.all import*
from geojson import Feature, Point, FeatureCollection, dump
from rasterio.plot import show
from pyproj import Transformer
from ipyleaflet import Map, basemaps, basemap_to_tiles

#### Initial Paths

In [3]:
IMAGES_PATH = Path('./data/islands/')
PRED_MASK_PATH = Path('./data/test_set/mask_pred')
PRED_MASK_PATH_TFMS = Path('./data/test_set/mask_pred_tfms/')

IMAGES_PATH.exists() == PRED_MASK_PATH.exists() == True

True

#### Resize Images (original .tif images - islands)

In order to extract the correct coordinates the original TIF images must be resized as well. 

Important:The actual coordinates are only stored in the TIF file, not in the PNGS or PNG maks. However, we need the pixel coordinates of the predictes PNG masks and then take the pixel at the same position in the TIF file (where the TIF has the EPSG:3857 coordinates stored).

In [4]:
def get_original_shape(item_name,org_dir):
    """assignes the TIF images as dataset"""
    for img in IMAGES_PATH.iterdir():
        if item_name == img.stem:
            with rio.open(img) as dataset: 
                return dataset.shape

Fastai comes in handy for the resizing process:

In [5]:
def resize_items(item,x,y): 
    resize_item = item.resize(torch.Size([x, y]))
    return resize_item

In [6]:
def threshold_item(item):
    "Convert all values between 0 and 255 to either 0 or 255"
    pred_msk_array_resized = array(item)
    return PILImage.create(pred_msk_array_resized.clip(0, 1).astype("uint8") * 255)

In [7]:
 def verify_shape_size(resized_pred_dir,org_dir):
    """ this function is vital, if shapes do not match, comparison is not possible"""
    print('Verifying shape sizes...')
    all_match = True
    for cnt,resized_msk in enumerate(PRED_MASK_PATH_TFMS.ls()):  
        for original_img in IMAGES_PATH.ls():           
            if resized_msk.stem[5:] == original_img.stem:
                resized_pred_msk_img = Image.open(resized_msk)    
                with rio.open(original_img) as dataset: 
                    if dataset.shape != resized_pred_msk_img.shape:
                        print(cnt, 'ATTENTION NOT SAME SHAPE!')
                        print('Resized predicted Mask: ', resized_msk.name, '  Shape: ', resized_pred_msk_img.shape)
                        print('Original Image: ', original_img.name, '  Shape: ', resized_pred_msk_img.shape, '\n')
                        all_match = False  
    
    if all_match:
        print('All shapes match!')   

### Stem is for item/file name without sufix

In [8]:
def align_pred_to_original_shape(pred_dir, org_dir, resized_pred_dir_out):
    
    # Check directory
    if not resized_pred_dir_out.exists():
        os.mkdir(resized_pred_dir_out)
    else:
        usr_input = input('Output folder already existis. Delete? (Y/N): ').upper()
        if usr_input == ('Y'):
            shutil.rmtree(resized_pred_dir_out)
            os.mkdir(resized_pred_dir_out)  
        else:
            print('Provide different output directory')
    print('Resizing predicted mask..')
    
    # Align pred mask shape with original image shape and save mask into new folder
    for pred_msk in pred_dir.ls():
        if os.path.isdir(pred_msk) == False:
            pred_msk_img = Image.open(pred_msk)          
            org_img_shape = get_original_shape(pred_msk.stem[5:],org_dir) # get every island and corresponding pred_mask image based on matching name
            pred_msk_img_resized= resize_items(item=pred_msk_img,x=org_img_shape[1],y=org_img_shape[0])
            pred_msk_img_resized_threshold=threshold_item(item=pred_msk_img_resized)
            pred_msk_img_resized_threshold.save(resized_pred_dir_out/pred_msk.name)
        else:
            print("Is a directory:")
            print(pred_msk)

    
    verify_shape_size(resized_pred_dir_out,org_dir)

In [ ]:
align_pred_to_original_shape(PRED_MASK_PATH,IMAGES_PATH,PRED_MASK_PATH_TFMS)

####  Map Coordinates and find centroid of individual objects

To enable an actual comparision with coordinates from OpenStreetMap, the centroid points of refuge islands must be calculated. Only then, the found coordinates of the model and the coordinates assigend to refuge island on OpenStreetMap can be compared.

In [31]:
def convert_coordinates(x,y):
    """
    EPSG:3857 --> Spherical Mercator projection coordinate 
    EPSG:4326 --> WGS84 
    """
    points = [(x,y)]
    transformer = Transformer.from_crs(3857, 4326)
    for pt in transformer.itransform(points): 
        return pt

In [32]:
def get_pixel_coordinates(pred_mask,output):
    crossings = list()
    
    # Convert Mask into 2dimensional array    
    gray_image = cv2.cvtColor(cv2.imread(str(pred_mask)) , cv2.COLOR_BGR2GRAY)

    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(image=gray_image,mode=cv2.RETR_TREE,method=cv2.CHAIN_APPROX_SIMPLE)

    for i,c in enumerate(contours):
        
        if cv2.contourArea(c) > 210: # Exclude small marks    
            crossing = np.zeros(gray_image.shape[:2], np.uint8)
            cv2.drawContours(crossing, [c], -1, (255), -1)
            loc, dims, angle = cv2.minAreaRect(c) 
    
            # Calculate moments for each contour
            M = cv2.moments(c)
            
            # Calculate x,y coordinate of center
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            
            # Display the image
            if output:
                print('crossing #',i+1, ' location ',' X: ', round(loc[0]),' Y: ',round(loc[1],))
            crossings.append([cX,cY])
    
    return crossings


In [37]:
def get_real_coordinates(data, output):    
    if output:
        print('EPGS:3857 ', data[0][1],data[0][0])
    x,y = convert_coordinates(data[0][1],data[0][0])        
    if output:
        print('WGS 84: ', x,y,'\n') 
    return x,y
                      

### Note:
Contrary to the usual form of geocoordinates (latitude then altitude), the order is reversed in a geojson file --> First altitude then latitude.

In [38]:
def get_geo_json(coordinates):    
    "Taskes list with tuple coordinates and creates a geojson"   
    
    feature_list = list()
    
    for x,y in coordinates:
        feature_list.append(Feature(geometry=Point((y, x))))
  
    feature_collection = FeatureCollection(feature_list)
    
    return feature_collection

In [39]:
def main(output):
    cnt = 1
    real_coordinates = list()
    print('Mappingprocess started..')
    # Get mask and real image
    for cnt,resized_msk in enumerate(PRED_MASK_PATH_TFMS.ls()):        
        for original_img in IMAGES_PATH.ls():           
            if resized_msk.stem[5:] == original_img.stem:
                with rio.open(original_img) as dataset: 
                    if output:
                        pyplot.imshow(dataset.read(4))
                        pyplot.show()
                        print(resized_msk.stem)
                        print('shape predicted: ', tensor(Image.open(resized_msk)).shape)
                        print('shape original: ', dataset.shape)
                    
                    # Get pixel coordinates 
                    crossings = get_pixel_coordinates(resized_msk,output)
                    
                    for crossing in crossings:
                        cX,cY = crossing
                        val = dataset.read(4)
                        no_data=dataset.nodata  
                        if val[cX,cY] != no_data:
                            data = [(dataset.xy(cY,cX)[1],dataset.xy(cY,cX)[0],val[cY,cX])]
                            real_coordinates.append((get_real_coordinates(data,output)))
        print('item ' + str(cnt) + '/' + str(len(PRED_MASK_PATH_TFMS.ls())))
        cnt +=1
    geo_json = get_geo_json(real_coordinates)
    
    with open('predicted_islands.geojson','w') as f:
        dump(geo_json,f)

    return geo_json
    

In [41]:
main(output=False)

Mappingprocess started..
item 0/637
item 1/637
item 2/637
item 3/637
item 4/637
item 5/637
item 6/637
item 7/637
item 8/637
item 9/637
item 10/637
item 11/637
item 12/637
item 13/637
item 14/637
item 15/637
item 16/637
item 17/637
item 18/637
item 19/637
item 20/637
item 21/637
item 22/637
item 23/637
item 24/637
item 25/637
item 26/637
item 27/637
item 28/637
item 29/637
item 30/637
item 31/637
item 32/637
item 33/637
item 34/637
item 35/637
item 36/637
item 37/637
item 38/637
item 39/637
item 40/637
item 41/637
item 42/637
item 43/637
item 44/637
item 45/637
item 46/637
item 47/637
item 48/637
item 49/637
item 50/637
item 51/637
item 52/637
item 53/637
item 54/637
item 55/637
item 56/637
item 57/637
item 58/637
item 59/637
item 60/637
item 61/637
item 62/637
item 63/637
item 64/637
item 65/637
item 66/637
item 67/637
item 68/637
item 69/637
item 70/637
item 71/637
item 72/637
item 73/637
item 74/637
item 75/637
item 76/637
item 77/637
item 78/637
item 79/637
item 80/637
item 81/637
i

{"features": [{"geometry": {"coordinates": [8.793246, 47.410943], "type": "Point"}, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [8.542755, 47.514511], "type": "Point"}, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [8.542838, 47.514682], "type": "Point"}, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [8.743609, 47.361766], "type": "Point"}, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [8.386937, 47.459774], "type": "Point"}, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [8.386698, 47.459889], "type": "Point"}, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [9.48283, 47.167813], "type": "Point"}, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [8.542314, 47.520267], "type": "Point"}, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [8.573818, 47.327384], "type": "Point"}, "properties": {}, "type": "Feature"}, {"geometry": {"